In [2]:
# Change directory to current

cd ~/GitHub/Clustering

SyntaxError: invalid syntax (<ipython-input-2-8631cf83f9b6>, line 3)

In [3]:
# Test import

import main, cfilter, suffix, scluster, edistance
reload(main)
reload(cfilter)
reload(suffix)
reload(scluster)
reload(edistance)

'import success'

'import success'

In [4]:
# Read raw data

from main import read

raw_candidate_industries = read('data/candidate_industries_04_10_2015.txt')
raw_job_industries       = read('data/job_industries_04_10_2015.txt')
raw_relevant_industries  = read('data/industry_of_relevant_users.txt')
raw_educations           = read('data/educ_fields_of_relevant_users.txt')

In [5]:
# Partner with clean data
# Tuple structure: (raw, clean)

from main import cleaners

def pair_with_clean(data, style='industry'):
    return zip(data, map(cleaners[style], data))

candidate_industries     = pair_with_clean(raw_candidate_industries)
job_industries           = pair_with_clean(raw_job_industries)
relevant_industries      = pair_with_clean(raw_relevant_industries)
educations               = pair_with_clean(raw_educations, style='course')

In [6]:
# Group unique entries for each clean data

from main import smash

def groupunique(datapair):
    
    # get clean part first
    cleanpart = zip(*datapair)[1]
    
    # smash into a unique set
    return smash(cleanpart)

candidates        = groupunique(candidate_industries)
jobs              = groupunique(job_industries)
relevant          = groupunique(relevant_industries)
courses           = groupunique(educations)

In [8]:
import suffix
reload(suffix)

scluster.lattice_spanning()

def all_matches(data):
    trie = suffix.Trie(courses)
    results = []
    for item in data:
        results.append(trie.search(item, 1))
    return results


%time all_matches(courses)

Wall time: 1min 13s


[[('5', 1),
  ('A', 1),
  ('B', 1),
  ('E', 1),
  ('H', 1),
  ('I', 1),
  ('N', 1),
  ('S', 1),
  ('U', 1),
  ('Y', 1)],
 [('MATHEMTICS', 0), ('MATHEMATICS', 1)],
 [('PAMANTASAN LUNGSOD NG MAKATI', 0)],
 [('LINGUISTICS', 0)],
 [('HOSPITALITY MANANGEMENT', 0), ('HOSPITALITY MANAGEMENT', 1)],
 [('FOOD SERVICE MANAGEMENT', 0)],
 [('ECONOMICS HONORS', 0)],
 [('COMPUTER SCIENCE INFORMATION TECHNOLOGY', 0),
  ('COMPUTER SCIENCES INFORMATION TECHNOLOGY', 1)],
 [('COMPUTER ELECTRONIC TECHNICIAN', 0)],
 [('MULTIMEDIA ARTS SCIENCES', 0)],
 [('I COMMERCE', 0), ('COMMERCE', 1)],
 [('SOCIAL SCIENCE HISTORY', 0)],
 [('ENGINEERING', 0)],
 [('BROADCASTING', 0)],
 [('ELECTRONIC COMMUNICATION ENGINEER', 0),
  ('ELECTRONICS COMMUNICATION ENGINEER', 1)],
 [('TRANSLATION', 0)],
 [('CHILDHOOD STUDIES', 0)],
 [('NURSING AIDE ING', 0)],
 [('BUSINESS INFORMATION SYSTEM', 0)],
 [('DENTAL MEDICINE', 0)],
 [('COMPUER SECRETATIAL OFFICE MANAGEMENT', 0)],
 [('WELDING SMAW GMAW', 0)],
 [('CULINARY FOOD BEVERAGES', 0

In [6]:
# Test: suffix trie
# with search function using edit distance
# O(knlogn) dynamic complexity

import suffix
reload(suffix)

%time trie = suffix.Trie(courses)
trie.insert('computer science 2')
trie.search('computer science 2', k=4)

Wall time: 303 ms


[('computer science 2', 0),
 ('COMPUTER SCIENCE', 1),
 ('COMPUTER SCIENCES', 1),
 ('COMPUTER SCIENCE W', 1),
 ('COMPUTER SCIEMCE', 2),
 ('COMPUTER SIENCE', 2),
 ('CUMPUTER SCIENCE', 2),
 ('BCS COMPUTER SCIENCE', 4),
 ('BCS COMPUTER SCIENCES', 4)]

In [7]:
# Test: suffix array

reload(suffix)

data = ['gattaca', 'atta', 'aca', 'tac']
%time sa = suffix.SuffixArray(data, ignore_spaces=False)

print '%s %s %25s\t   %10s [ID]\n' % ('Index', 'LCP', 'Concatenated Suffix', 'Original String')

for index in sa:
    print '[%-2d]' % index,
    print '  %d' % sa.lcp[index],
    print '\t%-25s' % sa.string[index:],

    string_index = sa.G[index]
    string_data = sa.data[string_index]

    print '\t%10s [%d]' % (string_data, string_index)

Wall time: 1e+03 µs
Index LCP       Concatenated Suffix	   Original String [ID]

[20]   0 	$                         	       tac [3]
[16]   0 	$tac$                     	       aca [2]
[12]   0 	$aca$tac$                 	      atta [1]
[7 ]   0 	$atta$aca$tac$            	   gattaca [0]
[15]   0 	a$tac$                    	       aca [2]
[11]   1 	a$aca$tac$                	      atta [1]
[6 ]   1 	a$atta$aca$tac$           	   gattaca [0]
[18]   1 	ac$                       	       tac [3]
[13]   2 	aca$tac$                  	       aca [2]
[4 ]   3 	aca$atta$aca$tac$         	   gattaca [0]
[8 ]   1 	atta$aca$tac$             	      atta [1]
[1 ]   4 	attaca$atta$aca$tac$      	   gattaca [0]
[19]   0 	c$                        	       tac [3]
[14]   1 	ca$tac$                   	       aca [2]
[5 ]   2 	ca$atta$aca$tac$          	   gattaca [0]
[0 ]   0 	gattaca$atta$aca$tac$     	   gattaca [0]
[10]   0 	ta$aca$tac$               	      atta [1]
[17]   2 	tac$                     

In [158]:
# Test: suffix array
# Concatenates all data into one big string
# O(nlogn) Complexity

reload(suffix)
%time sa = suffix.SuffixArray(jobs, ignore_spaces = True)


for i in sa:
    
    ID = sa.G[i]    
    print 'LCP:', sa.lcp[i], sa.string[i:i+30] + '...',
    print '\t', ID, sa.data[ID]

Wall time: 283 ms
LCP: 0 $... 	360 AMUSEMENT
LCP: 0 $AMUSEMENT$... 	359 MARKETING
LCP: 0 $MARKETING$AMUSEMENT$... 	358 CULINARY
LCP: 0 $CULINARY$MARKETING$AMUSEMENT$... 	357 PUBLIC RELATIONS
LCP: 0 $PUBLICRELATIONS$CULINARY$MARK... 	356 COMMERCIAL REAL ESTATE
LCP: 0 $COMMERCIALREALESTATE$PUBLICRE... 	355 DEVELOPMENT
LCP: 0 $DEVELOPMENT$COMMERCIALREALEST... 	354 DISTRIBUTOR
LCP: 0 $DISTRIBUTOR$DEVELOPMENT$COMME... 	353 SKILLED WORKER DRIVER
LCP: 0 $SKILLEDWORKERDRIVER$DISTRIBUT... 	352 TECHNICAL SUPPORT REPRESENTATIVE
LCP: 0 $TECHNICALSUPPORTREPRESENTATIV... 	351 COMPUTER SOFTWARE
LCP: 0 $COMPUTERSOFTWARE$TECHNICALSUP... 	350 MULTI LEVEL MARKETING
LCP: 0 $MULTILEVELMARKETING$COMPUTERS... 	349 EXAMINATION ADMINISTRATION DIVISION
LCP: 0 $EXAMINATIONADMINISTRATIONDIVI... 	348 MANPOWER SERVICE PROVIDER
LCP: 0 $MANPOWERSERVICEPROVIDER$EXAMI... 	347 INFORMATION TECHNOLOGY OUTSOURCING
LCP: 0 $INFORMATIONTECHNOLOGYOUTSOURC... 	346 MEDICAL
LCP: 0 $MEDICAL$INFORMATIONTECHNOLOGY... 	345 FURNITURE


In [159]:
# Test: similarity matrix
# returns

reload(suffix)
%time sm = sa.similarity_graph()

sm

Wall time: 17 ms


[((34, 294), 12),
 ((112, 297), 4),
 ((120, 217), 32),
 ((321, 357), 2),
 ((30, 287), 4),
 ((34, 153), 8),
 ((41, 57), 4),
 ((336, 337), 30),
 ((191, 232), 508),
 ((157, 349), 8),
 ((33, 343), 12),
 ((132, 218), 2),
 ((79, 253), 8),
 ((49, 58), 62),
 ((107, 185), 2),
 ((15, 143), 4),
 ((181, 248), 4),
 ((130, 272), 2),
 ((66, 264), 4),
 ((122, 278), 4),
 ((140, 153), 8),
 ((70, 94), 48),
 ((226, 312), 4),
 ((279, 294), 2),
 ((248, 352), 8),
 ((50, 234), 2),
 ((92, 158), 16),
 ((75, 254), 16),
 ((93, 289), 4),
 ((139, 142), 2),
 ((235, 315), 4),
 ((275, 279), 2),
 ((39, 349), 2),
 ((95, 256), 4),
 ((43, 339), 56),
 ((64, 252), 4),
 ((11, 297), 4),
 ((233, 302), 28),
 ((286, 288), 8),
 ((324, 328), 2),
 ((325, 326), 510),
 ((127, 179), 24),
 ((273, 352), 24),
 ((239, 256), 4),
 ((8, 94), 520192),
 ((46, 352), 4),
 ((88, 353), 4),
 ((282, 333), 16),
 ((7, 15), 2),
 ((28, 176), 4),
 ((331, 339), 2),
 ((158, 232), 4),
 ((78, 160), 8),
 ((153, 271), 260096),
 ((78, 124), 2),
 ((193, 352), 8)

In [160]:
import scluster
reload(scluster)
%time cluster = scluster.dense_spanning(jobs, min_common=20)

Wall time: 321 ms


[['INFORMATION TECHNOLOGY SOFTWARE',
  'INFORMATION TECHNOLOGY OR COMPUTER',
  'INFORMATION TECHNOLOGY',
  'INFORMATION TECHNOLOGY COMPANY',
  'INFORMATION TECHNOLOGY SERVICES',
  'INFORMATION TECHNOLOGY SOTFWARE',
  'INFORMATION TECHNOLOGY CONSULTING FIRM',
  'INFORMATION TECHNOLOGY OUTSOURCING'],
 ['BUSINESS PROCESS OUTSOURCING',
  'CREATIVE PROCESS OUTSOURCING',
  'RECRUITMENT PROCESS OUTSOURCING',
  'BUSINESS PROCESS OUTSOURCING CALL CENTER'],
 ['INDUSTRIAL ENGINEERING', 'MECHANICAL OR INDUSTRIAL ENGINEERING'],
 ['PLASTICS MANUFACTURING'],
 ['PRODUCT DEVELOPMENT'],
 ['BUSINESS SERVICES'],
 ['ELECTRONICAL'],
 ['RESEARCH'],
 ['COLLECTIONS DEPARTMENT'],
 ['POWER'],
 ['REAL ESTATE CONDOMINIUMS'],
 ['TAX'],
 ['LEISURE'],
 ['ENGINEERING'],
 ['SPA'],
 ['TECHNICAL SUPPORT DEPARTMENT'],
 ['RENTAL SERVICES'],
 ['AUDIT'],
 ['MARITIME'],
 ['PRODUCT MANAGEMENT'],
 ['HOSPITALITY'],
 ['LOGISTICS'],
 ['GUIDANCE'],
 ['MARKETING ANALYTICS'],
 ['PRIMARY'],
 ['WATER'],
 ['SAFETY OFFICER'],
 ['HEALTHCA

In [11]:
cluster

[['COMPUTER SCIENCE INFORMATION TECHNOLOGY',
  'COMPUTER ELECTRONIC TECHNICIAN',
  'ELECTRONIC COMMUNICATION ENGINEER',
  'BUSINESS INFORMATION SYSTEM',
  'COMPUER SECRETATIAL OFFICE MANAGEMENT',
  'ELECTRICAL COMPUTER ENGINEERING SIX YEAR CURRICULUM',
  'MARKETING ADVERTISING',
  'INTER DISCIPLINARY STUDIES',
  'INTERDISCIPLINARY STUDIES TRACKS IN MANAGEMENT COMMUNICATION',
  'INTERNATIONAL BUSINESS MARKETING',
  'SCIENCE INFORMATION TECHNOLOGY',
  'COMPUTER SECRETARIAL OFFICE MANAGEMENT',
  'BUSINESS ADMINISTRATION MAJOY IN BANKING FINANCE',
  'NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
  'FINANCIAL MANAGEMENT HOTEL RESTAURANT',
  'ELECTRONICS TELECOMMUNICATIONS TECHNOLOGY',
  'BUSINESS ADMINISTRATION MARKETING MANAGEMENT',
  'OFFICE INFORMATION SYSTEM',
  'INFORMATION COMMUNICATIONS TECHNOLOGY',
  'INFORMATION TECHNOLOGY IN PROGRAMMING',
  'COMPUTER SECRETARIAL BOOKKEEPING',
  'COMMUNICATIONS TECHNOLOGY MANAGEMENT',
  'INSTRUMENTAION CONTROL ENGINEERING TECHNOLOGY',
  'ELECTRONICS C

In [13]:
reload(scluster)
# remove noise
cl = scluster.lattice_spanning(courses, min_common = 3)
noise = []

def variance(data):
    if len(data) == 1: return 0
    mean = float(sum(data)) / len(data)
    var = sum(map(lambda x: (x - mean) ** 2, data)) / (len(data) - 1)
    print 'variance:', var, data[:10]
    return var

while variance(map(len,cl) + map(len,noise)) > 20:
    cl.sort(key=len, reverse=True)
    noise.extend(cl[1:])
    cl = scluster.spanning_forest(cl[0], n_clusters=len(cl[0]) / 8)


variance: 132194.314286 [1, 1, 1, 1, 2152, 1, 1, 1, 1, 1]
variance: 11490.990252 [1867, 3, 3, 3, 3, 2, 2, 2, 2, 2]
variance: 4826.64053345 [1608, 4, 4, 3, 3, 3, 3, 2, 2, 2]
variance: 2508.80995755 [1359, 4, 4, 4, 4, 3, 3, 3, 3, 3]
variance: 1386.62315809 [1120, 9, 6, 6, 5, 5, 4, 4, 4, 4]
variance: 769.091273156 [896, 17, 9, 5, 5, 4, 4, 4, 4, 4]
variance: 419.529089272 [696, 10, 8, 7, 5, 5, 5, 5, 5, 5]
variance: 242.514687571 [548, 20, 9, 9, 5, 4, 4, 4, 3, 3]
variance: 131.591604912 [413, 24, 15, 8, 6, 3, 3, 3, 3, 3]
variance: 62.9844735438 [286, 44, 12, 7, 6, 3, 3, 3, 3, 2]
variance: 22.0879489929 [131, 98, 14, 6, 2, 2, 2, 2, 2, 2]
variance: 18.6131510377 [112, 4, 2, 1, 1, 1, 1, 1, 1, 1]


In [14]:
cl

[['COMPUTER SCIENCE INFORMATION TECHNOLOGY',
  'ELECTRONIC COMMUNICATION ENGINEER',
  'SCIENCE INFORMATION TECHNOLOGY',
  'NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
  'ELECTRONICS TELECOMMUNICATIONS TECHNOLOGY',
  'INFORMATION COMMUNICATIONS TECHNOLOGY',
  'INFORMATION TECHNOLOGY IN PROGRAMMING',
  'COMMUNICATIONS TECHNOLOGY MANAGEMENT',
  'INSTRUMENTAION CONTROL ENGINEERING TECHNOLOGY',
  'ELECTRONICS COMMUNICATIONS',
  'INFORMATION TECHNOLOGY GAME DESIGN DEVELOPMENT',
  'ELECTRONICS COMMUNICATIONS ENGINEERING',
  'BUSINESS INFORMATION TECHNOLOGY',
  'INFORMATION TECHNOLOGY MANAGEMENT',
  'INFORMATION COMMUNICATION COMPUTER MANAGEMENT',
  'AVIATION INFORMATION TECHNOLOGY',
  'ELECTRINOCS COMMUNICATIONS ENGINEERING',
  'INFORMATION TECHNOLOGY COMPUTER',
  'ELECTRONICS COMMUNICATIONS ENGINEERING MANAGEMENT TECHNOLOGY',
  'MECHANICAL ENGINEERING TECHNO OGY',
  'MANUFACTURING ENGINEERING MANAGEMENT',
  'INFORMATION TECHNOLOGY MANAGEMENT SYSTEM',
  'LEGAL OFFICE MANAGEMENT',
  'CHEMICAL E

In [180]:
import scluster
reload(scluster)
%time dc = scluster.double_cluster(courses, min_common=5, step=2, eps=0.4, algorithm='lattice')

Wall time: 1min 5s


In [182]:
dc

[[[[[[[[[[[[['ELECTRONIC COMMUNICATION ENGINEER',
             'ECT ELECTRONIC COMMUNICATION TECHNICIAN',
             'ELECTRONIC COMMUNICATION ENGINEERING TECHNOLOGY',
             'ELECTRONIC COMMUNICATION ENGINEERING',
             'ELECTRONIC COMMUNICATION ENGINEERINH',
             'ELECTRONIC COMMUNICATION ENGINEER CABLE TELEVISION SATELLITE BROADCAST',
             'MECHANICAL PRODUCTION ENGINEERING TECHNOLOGY',
             'ELECTRONIC COMMUNICATION TECHNICIAN'],
            ['ELECTRONICS COMMUNICATIONS ENGINEERING',
             'ELECTRINOCS COMMUNICATIONS ENGINEERING',
             'ELECTRONICS COMMUNICATIONS ENGINEERING MANAGEMENT TECHNOLOGY',
             'ELECTRONIC COMMUNICATIONS ENGINEERING',
             'ELECTRONICS COMMUNICATIONS TECHNOLOGY',
             'ELECTRONICS COMMUNICATIONS ENGINEERING TECHNOLOGY',
             'COMPUTER ENGINEERING MANAGEMENT TECHNOLOGY'],
            [['NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
              'ELECTRONICS TELECOMMUNICATION

In [183]:
reload(scluster)
%time labels = scluster.label_heirarchy(dc)

--------------+ ELECTRONIC COMMUNICATION ENGINEER
--------------+ ECT ELECTRONIC COMMUNICATION TECHNICIAN
--------------+ ELECTRONIC COMMUNICATION ENGINEERING TECHNOLOGY
--------------+ ELECTRONIC COMMUNICATION ENGINEERING
--------------+ ELECTRONIC COMMUNICATION ENGINEERINH
--------------+ ELECTRONIC COMMUNICATION ENGINEER CABLE TELEVISION SATELLITE BROADCAST
--------------+ MECHANICAL PRODUCTION ENGINEERING TECHNOLOGY
--------------+ ELECTRONIC COMMUNICATION TECHNICIAN
-------------+ ELECTRONIC COMMUNICATION ENGINEERING
--------------+ ELECTRONICS COMMUNICATIONS ENGINEERING
--------------+ ELECTRINOCS COMMUNICATIONS ENGINEERING
--------------+ ELECTRONICS COMMUNICATIONS ENGINEERING MANAGEMENT TECHNOLOGY
--------------+ ELECTRONIC COMMUNICATIONS ENGINEERING
--------------+ ELECTRONICS COMMUNICATIONS TECHNOLOGY
--------------+ ELECTRONICS COMMUNICATIONS ENGINEERING TECHNOLOGY
--------------+ COMPUTER ENGINEERING MANAGEMENT TECHNOLOGY
-------------+ ELECTRONICS COMMUNICATIONS ENGINEERIN

In [171]:
labels

('A',
 2186,
 [('BUSI',
   [('MEDIA',
     [('ACTING',
       [('GENERAL',
         [('CATERING',
           [('MARKETING',
             [('AESTHETICS',
               [('MARINE ENGINE',
                 [('MARINE ENGINE',
                   [('MARINE ENGINE',
                     [('ANT MANAGEMENT',
                       [('INTERIOR DESIGN',
                         [('COMPUTER SCIENCES',
                           [('COMPUTER SCIENCES',
                             [('COMPUTER EDUCATION',
                               [('COMPUTER MANAGEMENT',
                                 [('COMPUTER ENGINEERING',
                                   [('BISINESS ADMINISTRATION',
                                     [('INFORMATION MANAGEMENT',
                                       [('ENGINEERING TECHNOLOGY',
                                         [('COMMUNICATION TECHNOLOGY',
                                           [('COMMUNICATIONS TECHNOLOGY',
                                             [(

In [178]:
names = set()
def deep(x):
    if isinstance(x[-1], list):
        names.add(x[0])
        for item in x[-1]:
            deep(item)
deep(labels)

In [179]:
len(names)

1844